In [ ]:
#Data Cleaning Section
import pandas as pd


In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,20)

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Dataset/Bengaluru_House_Data.csv")
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.groupby('area_type')['area_type'].agg('count')

In [ ]:
df2 = df1.drop(['area_type', 'society', 'balcony', 'availability'], axis = 'columns')

In [ ]:
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
df3 = df2.dropna()
df3.isnull().sum()

In [ ]:
df3.shape

In [ ]:
df3['size'].unique()

In [ ]:
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
df3.head()

In [ ]:
df3['bhk'].unique()

In [ ]:
df3[df3.bhk>20]

In [ ]:
df3.total_sqft.unique()

In [ ]:
def is_float(x):
  try:
    float(x)
  except:
    return False
  return True



In [ ]:
df3[~df3['total_sqft'].apply(is_float)].head()

In [ ]:
 def convert_sqft_to_num(x):
  tokens = x.split('-')
  if len(tokens) == 2:
    return(float(tokens[0])+ float(tokens[1]))/2
  try:
    return float(x)
  except:
    return None

In [ ]:
convert_sqft_to_num('2166')

In [ ]:
df4 = df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_sqft_to_num)

In [ ]:
df4.head()

In [ ]:
df4.loc[30]

In [ ]:
(2100+2850)/2
#Data Cleaning section completes here

In [ ]:
#Feature Engineering
df4.head()

In [ ]:
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000/df5['total_sqft']

In [ ]:
df5.head()

In [ ]:
len(df5.location.unique())

In [ ]:
df5.location = df5.location.apply(lambda x: x.strip())
location_stats = df5.groupby('location')['location'].agg('count').sort_values(ascending = False)
location_stats

In [ ]:
len(location_stats[location_stats<=10])


In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
len(df5.location.unique())

In [ ]:
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

In [ ]:
#Feature Engineering Completes here
df5.head(10)

In [ ]:
#Outlier Removal
df5[df5.total_sqft/df5.bhk<300].head()

In [ ]:
df5.shape

In [ ]:
df6 = df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape

In [ ]:
df6.price_per_sqft.describe()

In [ ]:
def remove_apps_outliers(df):
  df_out = pd.DataFrame()
  for key, subdf in df.groupby('location'):
      m= np.mean(subdf.price_per_sqft)
      st = np.std(subdf.price_per_sqft)
      reduced_df = subdf[(subdf.price_per_sqft > (m-st)) & (subdf.price_per_sqft <= (m+st))]
      df_out = pd.concat([df_out, reduced_df], ignore_index= True)
  return df_out
df7 = remove_apps_outliers(df6)
df7.shape

In [ ]:
def plot_scatter_chart(df, location):
    bhk2 = df[(df.location==location) & (df.bhk == 2)]
    bhk3 = df[(df.location==location) & (df.bhk == 3)]

    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft, bhk2.price_per_sqft, color = 'blue', label = '2 BHK ', s = 50)
    plt.scatter(bhk3.total_sqft, bhk3.price_per_sqft, marker = '+', color = 'green', label = '3 BHK', s = 50)
    plt.xlabel = ("Total Square Feet Area ")
    plt.ylabel = ("Price Per Square Feet ")
    plt.title(location)
    plt.legend()

plot_scatter_chart(df7, "Rajaji Nagar")

In [ ]:
def remove_bhk_outlier(df):
  exclude_indices = np.array([])
  for location, location_df in df.groupby('location'):
    bhk_stats ={}
    for bhk, bhk_df in location_df.groupby('bhk'):
      bhk_stats[bhk] = {
          'mean': np.mean(bhk_df.price_per_sqft),
          'std': np.std(bhk_df.price_per_sqft),
           'count': bhk_df.shape[0]
      }
    for bhk, bhk_df in location_df.groupby('bhk'):
      stats = bhk_stats.get(bhk-1)
      if stats and stats['count']>5:
        exculde_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
  return df.drop(exclude_indices, axis = 'index')

In [ ]:
df8 = remove_bhk_outlier(df7)
df8.shape

In [ ]:
plot_scatter_chart(df7, "Hebbal")

In [ ]:
import matplotlib.pyplot as plt
matplotlib.rcParams["figure.figsize"] = (20, 20)
plt.hist(df8.price_per_sqft,rwidth = 0.8)
plt.xlabel = ("Price Per Square Feet")
plt.ylabel = ("Count")

In [ ]:
df8.bath.unique()

In [ ]:
df8[df8.bath>10]

In [ ]:
df8[df8.bath>df8.bhk+2]

In [ ]:
df9 = df8[df8.bath<df8.bhk+2]

In [ ]:
df9.shape

In [ ]:
df10 = df9.drop(['size', 'price_per_sqft'], axis = 'columns')
df10.head(3)
#Outlier Removal Section Complete

In [ ]:
#Model Secion
dummies = pd.get_dummies(df10.location)
dummies.head(3)

In [ ]:
df11 = pd.concat([df10, dummies.drop('other', axis = 'columns')], axis = 'columns')
df11.head(3)

In [ ]:
df12 = df11.drop('location', axis = 'columns')
df12.head(2)

In [ ]:
df12.shape

In [ ]:
X = df12.drop('price', axis = 'columns')
X.head()

In [ ]:
y = df12.price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train, y_train)
lr_clf.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv = ShuffleSplit(n_splits = 5, test_size = 0.2,random_state = 0)
cross_val_score(LinearRegression(), X, y, cv = cv)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso  # Corrected import
from sklearn.tree import DecisionTreeRegressor
def find_best_model_using_gridserch(X,y):
  algos = {
      'linear_regression' : {
          'model': LinearRegression(),
           'params': {
                'fit_intercept': [True, False],
                'n_jobs': [1, -1],
                'positive': [True, False]
            }
      },
      'lasso' : {
          'model': Lasso(),
          'params': {
              'alpha': [1, 2],
              'selection': ['random', 'cyclic']
          }
      },
      'decision_tree' : {
          'model': DecisionTreeRegressor(),
             'params': {
                'criterion': ['mse', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
      }
  }

  scores = []
  cv = ShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)
  for algo_name, config in algos.items():
      gs = GridSearchCV(config['model'], config['params'], cv = cv, return_train_score = False)
      gs.fit(X_train,y_train)
      scores.append({
          'model': algo_name,
          'best_score': gs.best_score_,
          'best_params': gs.best_params_
      })
  return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])



In [ ]:
find_best_model_using_gridserch(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _valida

,model,best_score,best_params
0,linear_regression,0.822349,"{'fit_intercept': True, 'n_jobs': 1, 'positive..."
1,lasso,0.672800,"{'alpha': 1, 'selection': 'random'}"
2,decision_tree,0.667684,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


In [ ]:
X.columns

Index(['total_sqft', 'bath', 'bhk', '1st Block Jayanagar',
       '1st Phase JP Nagar', '2nd Phase Judicial Layout',
       '2nd Stage Nagarbhavi', '5th Block Hbr Layout', '5th Phase JP Nagar',
       '6th Phase JP Nagar',
       ...
       'Vijayanagar', 'Vishveshwarya Layout', 'Vishwapriya Layout',
       'Vittasandra', 'Whitefield', 'Yelachenahalli', 'Yelahanka',
       'Yelahanka New Town', 'Yelenahalli', 'Yeshwanthpur'],
      dtype='object', length=244)

In [ ]:
def predict_price(location, sqft, bath, bhk):
  loc_index = np.where(X.columns==location)[0][0]


  x = np.zeros(len(X.columns))
  x[0] = sqft
  x[1] = bath
  x[2] = bhk
  if loc_index >= 0:
    x[loc_index] = 1

  return lr_clf.predict([x])[0]


In [ ]:
predict_price('1st Phase JP Nagar', 1000, 2, 2)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


86.33120168526462

In [ ]:
predict_price('1st Phase JP Nagar', 1000, 3, 3)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


87.88509323995571

In [ ]:
predict_price('Indira Nagar', 1000, 2, 2)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


168.6175952187994

In [ ]:
predict_price('Indira Nagar', 1000, 3, 3)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


170.17148677349053

In [ ]:
import pickle
with open('banglore_home_prices_model.pickle', 'wb') as f:
  pickle.dump(lr_clf, f)

In [ ]:
import json
columns = {
    'data_columns' : [col.lower()for col in X.columns]
}
with open("columns.json", "w") as f:
  f.write(json.dumps(columns))
  #Model Selecion complete

In [ ]:
#Flask Server
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=206e90d5fb340669678a66ff60323c56a76a17d788718aa88ed4f3a06d2fa8d8
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
port_no = 5000

In [ ]:
app = Flask(__name__)
ngrok.set_auth_token("2UeeBP6mIdpzJsJXrWGuhSPrFHV_3rZEdLoSg3xFXvjzMQT9B")
public_url = ngrok.connect(port_no).public_url
@app.route("/")
def home():
  return f"Running Flask On Google Colab!"
print(f"To access the Colab please click the link {public_url}")
app.run(port=port_no)

To access the Colab please click the link https://b3ef-104-196-26-85.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
